In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from warnings import filterwarnings
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from tqdm import tqdm
filterwarnings('ignore')

Cleaning

In [2]:
select_customers = False
cus_low = 19
cus_high = 36
trans_week_low = 12
trans_week_high = 20
val_week = 11
channel = 2

articles = pd.read_csv("/content/drive/MyDrive/cis520-final project/articles.csv")
customers = pd.read_csv("/content/drive/MyDrive/cis520-final project/customers.csv")
transactions = pd.read_csv("/content/drive/MyDrive/cis520-final project/transactions_train.csv")

articles = articles.dropna()
articles["text"] = articles["prod_name"].map(str) + " " + articles["product_type_name"] +" "+ articles["product_group_name"]+ " "+ articles['graphical_appearance_name']+" "+ articles['colour_group_name'] +" "+ articles['perceived_colour_value_name']+ " " + articles["perceived_colour_master_name"] +" "+ articles["department_name"]+ " "+ articles['index_name']+" "+articles['index_group_name'] +" "+articles['section_name']+ " "+ articles['garment_group_name']+" "+articles['detail_desc']

articles = articles.reset_index().drop('index',axis=1)

transactions = transactions[(transactions["article_id"].isin(articles["article_id"].unique()))&(transactions["sales_channel_id"]==channel)].reset_index(drop=True)

#encode
article_ids = np.unique(articles["article_id"].values)

le_article = LabelEncoder()
le_article.fit(article_ids)
articles["article_id"] = le_article.transform(articles["article_id"])

customer_ids = np.unique(customers["customer_id"].values)

le_customer = LabelEncoder()
le_customer.fit(customer_ids)
customers["customer_id"] = le_customer.transform(customers["customer_id"])

transactions = transactions.dropna()
transactions["customer_id"] = le_customer.transform(transactions["customer_id"])
transactions =  transactions.sort_values(by='customer_id')
transactions["article_id"] = le_article.transform(transactions["article_id"])
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])
transactions["week"] = (transactions["t_dat"].max() - transactions["t_dat"]).dt.days // 7

# if we want to select a certain age
if select_customers:
  customers = customers[(customers['age']>=cus_low)&(customers['age']<=cus_high)]
  transactions = transactions.merge(customers['customer_id'],on = 'customer_id',how='inner')

val = transactions[transactions['week']==val_week].reset_index()
val_cus = val.groupby('customer_id', sort=False)['article_id'].apply(list).reset_index()
transactions = transactions[(transactions['week']>=trans_week_low)&(transactions['week']<=trans_week_high)].reset_index()
transactions = transactions.drop(['index'],axis=1)
popularity = list(pd.DataFrame(transactions.groupby('article_id',sort=True)['customer_id'].count()).sort_values(by='customer_id',ascending=False)[:12].index)

df_all = articles[['article_id', 'product_code', 'text']]
merged_df = df_all.merge(transactions, how = 'inner', on = ['article_id'])
merged_df2 = merged_df.groupby('customer_id', sort=False)['text'].apply(' '.join).reset_index()
df_all_new = df_all[df_all["article_id"].isin(transactions["article_id"].unique())]

In [21]:
len(df_all_new)

25567

In [3]:
pop_trans = transactions[(transactions['week']>=12)&(transactions['week']<=13)].reset_index()
popularity = list(pd.DataFrame(pop_trans.groupby('article_id',sort=True)['customer_id'].count()).sort_values(by='customer_id',ascending=False)[:12].index)
popularity

[61202,
 94985,
 84579,
 92967,
 85823,
 1722,
 96879,
 22444,
 86147,
 22437,
 98404,
 84540]

Validation Criterion

In [4]:
def calc_map(y_true, y_score, k=None):
    if k is None: k=len(y_score)
    relevant = np.isin(y_score[:k], y_true) # relevant[i]==1 if y_score[i] is correct
    patk = np.cumsum(relevant)/np.arange(1, len(y_score[:k])+1) # patk[0]==P@1, patk[1]==P@2, ...
    return(np.sum(patk*relevant)/min(len(y_true), k)) # as defined by the competition

target = dict(zip(val_cus['customer_id'],val_cus['article_id']))

def recommendation_product(top, df_all):
  recommendation = pd.DataFrame(columns = ['customer_id', 'article_id'])
  count = 0
  for i in top:
      recommendation.at[count, 'customer_id'] = u
      recommendation.at[count, 'article_id'] = df_all['article_id'][i]
      count += 1
  return list(recommendation['article_id'])

TFIDF+KMeans

In [5]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_article = tfidf_vectorizer.fit_transform((df_all_new['text'])) 
tfidf_article_embed = tfidf_article.toarray()

In [ ]:
from sklearn import decomposition
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
pca.fit(tfidf_article_embed)
tfidf_article_PCA = pca.transform(tfidf_article_embed)

In [ ]:
np.sum(pca.explained_variance_)

0.6244307342992385

In [6]:
num_clusters = len(articles)//100
max_iter = 300
n_runs = 5

kmeans = KMeans(
    n_clusters=num_clusters, init='random',
    n_init=n_runs, max_iter=max_iter, 
    tol=1e-04, random_state=0,verbose=1
)

y_pred_km = kmeans.fit_predict(tfidf_article_embed)
df_all_new['KMeans-Cate-TFIDF']=y_pred_km

category_tf = df_all_new.groupby(['KMeans-Cate-TFIDF'])['article_id'].apply(list)

# get all the products in the category if the customer once bought product in this category
pred={}
for i, u in enumerate(tqdm(val_cus['customer_id'].unique())):
  if u in transactions['customer_id'].unique():
    cate_no = df_all_new[df_all_new['article_id'].isin((transactions['article_id'][transactions['customer_id']==u]).unique())]['KMeans-Cate-TFIDF'].unique()
    pred[u]=[]
    for j in [category_tf[i] for i in cate_no]:
      pred[u].extend(j)
  else:
    pred[u]=popularity

np.save("/content/drive/MyDrive/cis520-final project/text_tfidf_kmeans.npy",pred)

# each cluster has about 100 items
map=0
for u in val_cus['customer_id'].unique():
  map+=calc_map(np.array(target[u]), np.array(pred[u]) ,k=12)

map/len(val_cus['customer_id'].unique())

Initialization complete
Iteration 0, inertia 19276.269208855927
Iteration 1, inertia 11271.177343224499
Iteration 2, inertia 10820.413048658287
Iteration 3, inertia 10662.868384913127
Iteration 4, inertia 10587.680166097509
Iteration 5, inertia 10540.337693486232
Iteration 6, inertia 10513.454721525484
Iteration 7, inertia 10497.969826486787
Iteration 8, inertia 10489.257449134877
Iteration 9, inertia 10484.9044076191
Iteration 10, inertia 10481.017881732812
Iteration 11, inertia 10478.398897465298
Iteration 12, inertia 10476.72030825205
Iteration 13, inertia 10475.610271309819
Iteration 14, inertia 10475.273734608327
Iteration 15, inertia 10475.13374588399
Iteration 16, inertia 10474.937436756323
Iteration 17, inertia 10474.825323911838
Converged at iteration 17: strict convergence.
Initialization complete
Iteration 0, inertia 19286.30365186644
Iteration 1, inertia 11311.53074588401
Iteration 2, inertia 10902.234104308876
Iteration 3, inertia 10743.15987337855
Iteration 4, inertia 106

100%|██████████| 48184/48184 [17:56<00:00, 44.77it/s]


0.004806504134040924

In [ ]:
np.load('/content/drive/MyDrive/cis520-final project/text_tfidf_kmeans.npy',allow_pickle=True).item()

In [26]:
num_clusters = len(articles)//20
max_iter = 300
n_runs = 5

kmeans = KMeans(
    n_clusters=num_clusters, init='random',
    n_init=n_runs, max_iter=max_iter, 
    tol=1e-04, random_state=0,verbose=1
)

y_pred_km = kmeans.fit_predict(tfidf_article_embed)
df_all_new['KMeans-Cate-TFIDF']=y_pred_km

category_tf = df_all_new.groupby(['KMeans-Cate-TFIDF'])['article_id'].apply(list)

# get all the products in the category if the customer once bought product in this category
pred={}
for i, u in enumerate(tqdm(val_cus['customer_id'].unique())):
  if u in transactions['customer_id'].unique():
    cate_no = df_all_new[df_all_new['article_id'].isin((transactions['article_id'][transactions['customer_id']==u]).unique())]['KMeans-Cate-TFIDF'].unique()
    pred[u]=[]
    for j in [category_tf[i] for i in cate_no]:
      pred[u].extend(j)
  else:
    pred[u]=popularity

np.save("/content/drive/MyDrive/cis520-final project/text_tfidf_kmeans_new.npy",pred)

map=0
for u in val_cus['customer_id'].unique():
  map+=calc_map(np.array(target[u]), np.array(pred[u]) ,k=12)

map/len(val_cus['customer_id'].unique())

Initialization complete
Iteration 0, inertia 10753.98291463775
Iteration 1, inertia 6325.175563932076
Iteration 2, inertia 5986.3686742135615
Iteration 3, inertia 5856.355777017052
Iteration 4, inertia 5813.146468887306
Iteration 5, inertia 5794.889456732086
Iteration 6, inertia 5789.5010870704855
Iteration 7, inertia 5787.06971494609
Iteration 8, inertia 5785.7777985043795
Iteration 9, inertia 5785.480426394151
Converged at iteration 9: strict convergence.
Initialization complete
Iteration 0, inertia 10847.776167519643
Iteration 1, inertia 6327.200354361927
Iteration 2, inertia 5986.1333001012745
Iteration 3, inertia 5869.904276825722
Iteration 4, inertia 5832.26910070319
Iteration 5, inertia 5820.2990866421
Iteration 6, inertia 5815.005519992556
Iteration 7, inertia 5813.5310602517875
Iteration 8, inertia 5813.354583221917
Iteration 9, inertia 5812.921518754197
Iteration 10, inertia 5812.656539530463
Converged at iteration 10: strict convergence.
Initialization complete
Iteration 0, 

100%|██████████| 48184/48184 [17:57<00:00, 44.70it/s]


0.009992798538271976

TFIDF+Cosine Similarity

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_article = tfidf_vectorizer.fit_transform((df_all_new['text'])) 
tfidf_article_embed = tfidf_article.toarray()

pred_cs = {}

for u in tqdm(val_cus['customer_id'].unique()):
  if u in merged_df2['customer_id'].unique():
    index = np.where(merged_df2['customer_id'] == u)[0][0]
    cust_q = merged_df2.iloc[[index]]
    user_tfidf = tfidf_vectorizer.transform(cust_q['text'])
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_article)
    output2 = list(cos_similarity_tfidf)
    top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:12]
    tf_list_scores = [output2[i][0][0] for i in top]
    pred_cs[u] = recommendation_product(top, df_all)
  else:
    pred_cs[u] = popularity

map=0

for u in pred.keys():
  map+=calc_map(np.array(target[u]), np.array(pred_cs[u]), k=12)

map/len(pred_cs.keys())

In [ ]:
np.save("/content/drive/MyDrive/cis520-final project/tfidf_cs.npy",pred_cs)

In [ ]:
pred_cs = np.load("/content/drive/MyDrive/cis520-final project/tfidf_cs.npy")

BERT+KMeans

In [15]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
from tqdm import tqdm

## for processing
import re
import nltk

## for BERT
import torch
!pip install transformers
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

all = np.zeros([len(df_all_new['text']),768])
for i in tqdm(range((len(df_all_new['text'])//100)+1)):
  inputs = tokenizer(list(df_all_new['text'][100*i:100*(i+1)]), return_tensors="pt", padding=True,truncation=True,max_length=50, add_special_tokens = True,verbose=0) 
  bert_outputs = bert(**inputs)
  all[100*i:100*(i+1)] = bert_outputs.pooler_output.detach().numpy()


num_clusters = len(articles)//20
max_iter = 300
n_runs = 5

kmeans_bert = KMeans(
    n_clusters=num_clusters, init='random',
    n_init=n_runs, max_iter=max_iter, 
    tol=1e-04, random_state=0, verbose=1
)

y_pred_bert = kmeans_bert.fit_predict(all)
df_all_new['KMeans-Cate-bert']=y_pred_bert

category_bert = df_all_new.groupby(['KMeans-Cate-bert'])['article_id'].apply(list)

pred_bert={}
for i, u in enumerate(tqdm(val_cus['customer_id'].unique())):
  if u in transactions['customer_id'].unique():
    cate_no = df_all_new[df_all_new['article_id'].isin((transactions['article_id'][transactions['customer_id']==u]).unique())]['KMeans-Cate-bert'].unique()
    pred_bert[u]=[]
    for j in [category_bert[i] for i in cate_no]:
      pred_bert[u].extend(j)
  else:
    pred_bert[u]=popularity

np.save("/content/drive/MyDrive/cis520-final project/text_bert_kmeans_new.npy",pred_bert)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 256/256 [27:03<00:00,  6.34s/it]


Initialization complete
Iteration 0, inertia 41589.78296874756
Iteration 1, inertia 27141.66729381571
Iteration 2, inertia 26014.77748245198
Iteration 3, inertia 25672.977293658216
Iteration 4, inertia 25532.868349726326
Iteration 5, inertia 25482.043960805724
Iteration 6, inertia 25461.670680825784
Iteration 7, inertia 25455.540253676732
Iteration 8, inertia 25453.254055798687
Iteration 9, inertia 25453.040827282974
Converged at iteration 9: strict convergence.
Initialization complete
Iteration 0, inertia 41832.52436454077
Iteration 1, inertia 27311.8516351394
Iteration 2, inertia 26091.957387256112
Iteration 3, inertia 25707.240937754454
Iteration 4, inertia 25576.28534898362
Iteration 5, inertia 25527.35963819918
Iteration 6, inertia 25504.353879334147
Iteration 7, inertia 25493.905889684742
Iteration 8, inertia 25487.108764880257
Iteration 9, inertia 25485.222196500476
Iteration 10, inertia 25483.24695592011
Iteration 11, inertia 25482.773432743168
Iteration 12, inertia 25482.36575

100%|██████████| 48184/48184 [18:35<00:00, 43.18it/s]


TypeError: ignored

In [16]:
# 5256 center
map=0
for u in val_cus['customer_id'].unique():
  map+=calc_map(np.array(target[u]), pred_bert[u], k=12)

map/len(val_cus['customer_id'].unique())

0.0084510530992916

In [14]:
# 2102 center
pred_bert = np.load("/content/drive/MyDrive/cis520-final project/text_bert_kmeans.npy",allow_pickle=True).item()

map=0
for u in val_cus['customer_id'].unique():
  map+=calc_map(np.array(target[u]), pred_bert[u], k=12)

map/len(val_cus['customer_id'].unique())

0.004840219269592577

combining the recall results

In [ ]:
#tabular-kmeans-result
a = np.load('/content/drive/MyDrive/cis520-final project/tabular_kmeans.npy',allow_pickle=True).item()

#text-bert-kmeans-result
b = np.load('/content/drive/MyDrive/cis520-final project/text_bert_kmeans.npy',allow_pickle=True).item()

#text-tfidf-kmeans-result
c = np.load('/content/drive/MyDrive/cis520-final project/text_tfidf_kmeans.npy',allow_pickle=True).item()

In [ ]:
len(a.keys())

48192

In [ ]:
d = {}
d.update(a)
d.update(b)
d.update(c)

In [ ]:
for key in d:
  d[key] = set(d[key])

In [ ]:
m = list(val_cus['customer_id'].unique())
n = list(transactions['customer_id'].unique())
for i, u in enumerate(tqdm(m)):
  if u not in n:
    d[u] = popularity

100%|██████████| 48184/48184 [04:41<00:00, 171.07it/s]


In [ ]:
output = {}
for u in tqdm(m):
  key = le_customer.inverse_transform(np.array(u).reshape(-1))
  output[key[0]] = list(le_article.inverse_transform(list(d[u])))

100%|██████████| 48184/48184 [1:01:51<00:00, 12.98it/s]


In [ ]:
np.save('/content/drive/MyDrive/cis520-final project/text+tabular_kmeans.npy',output)